# CS 773 Data Mining and Security

## Course Project - Analyzing Open University Learning Analytics Dataset 

### Problem Statement 

Based on the given data of students enrolled in various courses identify the student who is at-risk so that suitable interventions can be taken to help students succeed. 

***About Dataset ------- Part-1 : Reading Data***

- Open University Learning Analytics Dataset (OULAD) contains data about courses, students and their interactions with Virtual Learning Environment (VLE) for seven selected courses (called modules). Presentations of courses start in February and October - they are marked by "B" and "J" respectively. The dataset consists of tables connected using unique identifiers. Dataset is stored in several csv files.  
- More information, examples and news can be found at: 
      https://analyse.kmi.open.ac.uk/open_dataset

Number of Instances: 
    - students in courses : 32953
    - course-presentations: 22
    - VLE pages           : 6364
    - VLE log entries     : 10655280
    - registration entries: 32953
    - assessments         : 206
    - assessment entries  : 173912


In [2]:
#importing Libraries
import numpy as np
import pandas as pd
import time

In [3]:
# Reading Data
student_info = pd.read_csv('Data/studentInfo.csv')
student_assessment = pd.read_csv('Data/studentAssessment.csv')
student_registration = pd.read_csv('Data/studentRegistration.csv')
student_vle = pd.read_csv('Data/studentVle.csv')
assessments = pd.read_csv('Data/assessments.csv')
courses = pd.read_csv('Data/courses.csv')
vle = pd.read_csv('Data/vle.csv')

#check for the data
print('----- Rows -----')
print('Courses     : ',courses.shape[0])
print('Assessments : ',assessments.shape[0])
print('vle(Virtual Learning Environment) : ',vle.shape[0])
print('Student Info         : ',student_info.shape[0])
print('student Assessment   : ',student_assessment.shape[0])
print('student Registration : ',student_registration.shape[0])
print('student Vle          : ',student_vle.shape[0])

----- Rows -----
Courses     :  22
Assessments :  206
vle(Virtual Learning Environment) :  6364
Student Info         :  32593
student Assessment   :  173912
student Registration :  32593
student Vle          :  10655280


In [4]:
### Lets see what is there in very file
# ---- For COURSES
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


***code_module*** : code name of the module, which serves as the identifier

***code_presentation*** : code name of the presentation. It consists of the year and “B” for the presentation starting in February and “J” for the presentation starting in October

***module_presentation_length*** :  length of the module-presentation in days

In [5]:
# ---- For Assessments
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


***code_module*** : identification code of the module,to which the assessment belongs.

***code_presentation*** : identification code of the presentation, to which the assessment belongs.

***id_assessment*** : identification number of the assessment

***assessment_type*** : type of assessment

***date*** : information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero)

***weight*** : weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%

In [6]:
# ---- For VLE - Virtual Learning Environment
vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,?,?
1,546712,AAA,2013J,oucontent,?,?
2,546998,AAA,2013J,resource,?,?
3,546888,AAA,2013J,url,?,?
4,547035,AAA,2013J,resource,?,?


***id_site*** : an identification number of the material

***code_module*** : an identification code for module

***code_presentation*** : the identification code of presentation

***activity_type*** : the role associated with the module material

***week_from*** : the week from which the material is planned to be used

***week_to*** : week until which the material is planned to be used

In [7]:
# ---- Students Info
student_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


***code_module*** : An identification code for a module on which the student is registered.

***code_presentation*** : The identification code of the presentation during which the student is registered on the module.

***id_student*** : A unique identification number for the student.

***gender*** : The student’s gender.

***region*** : Identifies the geographic region, where the student lived while taking the module-presentation.

***highest_education*** : Highest student education level on entry to the module presentation.

***imd_band*** : Specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.

***age_band*** : Band of the student’s age.

***num_of_prev_attempts*** : The number times the student has attempted this module.

***studied_credits*** : The total number of credits for the modules the student is currently studying.

***disability*** : Indicates whether the student has declared a disability

***final_result*** : student’s final result in the module-presentation

In [8]:
# ---- For Student assessments
student_assessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78
1,1752,28400,22,0,70
2,1752,31604,17,0,72
3,1752,32885,26,0,69
4,1752,38053,19,0,79


***id_assessment*** : the identification number of the assessment.

***id_student*** : a unique identification number for the student.

***date_submitted*** : the date of student submission, measured as the number of days since the start of the module presentation.

***is_banked*** : a status flag indicating that the assessment result has been transferred from a previous presentation.
   
***score*** : the student’s score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.

In [9]:
# ---- For Student Registrations
student_registration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159,?
1,AAA,2013J,28400,-53,?
2,AAA,2013J,30268,-92,12
3,AAA,2013J,31604,-52,?
4,AAA,2013J,32885,-176,?


***code_module*** : an identification code for a module

***code_presentation*** : The identification code of the presentation

***id_student*** : a unique identification number for the student

***date_registration (Days)*** : The date of student’s registration on the module presentation, this is the number of days measured relative to the start of the module-presentation 
(e.g. the negative value -30 means that the student registered to module presentation 30 days before it started)
   
***date_unregistration (Days)*** : date of student unregistration from the module presentation, this is the number of days measured relative to the start of the module-presentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file

In [10]:
# ---- student_vle
student_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


***code_module***: an identification code for a module

***code_presentation*** : the identification code of the module presentation

***id_student*** : a unique identification number for the student

***id_site*** : an identification number for the VLE material

***date (Days)*** : the date of student’s interaction with the material measured as the number of days since the start of the module-presentation

***sum_click*** : the number of times a student interacts with the material in that day

In [40]:
#get number of students registered for each course in every year
student_registration.groupby(['code_module','code_presentation']).size().reset_index(name='Students_enrolled')

,code_module,code_presentation,Students_enrolled
0,AAA,2013J,383
1,AAA,2014J,365
2,BBB,2013B,1767
3,BBB,2013J,2237
4,BBB,2014B,1613
5,BBB,2014J,2292
6,CCC,2014B,1936
7,CCC,2014J,2498
8,DDD,2013B,1303
9,DDD,2013J,1938


In [41]:
#check total assessments in each courses
assessments.groupby(['code_module','code_presentation']).size().reset_index(name='Total_assessments')

,code_module,code_presentation,Total_assessments
0,AAA,2013J,6
1,AAA,2014J,6
2,BBB,2013B,12
3,BBB,2013J,12
4,BBB,2014B,12
5,BBB,2014J,6
6,CCC,2014B,10
7,CCC,2014J,10
8,DDD,2013B,14
9,DDD,2013J,7


In [45]:
student_info['region'].value_counts()

Scotland                3446
East Anglian Region     3340
London Region           3216
South Region            3092
North Western Region    2906
West Midlands Region    2582
South West Region       2436
East Midlands Region    2365
South East Region       2111
Wales                   2086
Yorkshire Region        2006
North Region            1823
Ireland                 1184
Name: region, dtype: int64

### Initial Assumptions --- Path Forward

As we have to predict the at risk student, there are many features that might contribute to a student success, to extract those features we perform some Exploratory Data Analysis (EDA). 

Some initial assumptions can be made from the real world scenarios in order to move forward.

-> Students success is determined by the assessments i.e if he/she is submitting the assignments on time and how much score is obtained in every assessment by calculating a weighted average of those data we can determine whether or not a student is at risk.

->  Do we need to build a model for each course ???

-> need to check max courses a student can take in one semester

->get to know more about the students who withdraw from their courses

***A) Analyzing Student_info Data*** 

In [83]:
#get results of students
print('----------- Student Final Results -------------')
print(student_info['final_result'].value_counts())

#get student past credits
print('----------- Student previous credits -------------')
print(student_info['studied_credits'].value_counts())

#get student with max and min credits
print('----------- Student Previous Credits Details-------------')
print('Minimum_Credits : ',str(student_info['studied_credits'].min()))
print('Maximum_Credits: ',str(student_info['studied_credits'].max()))
print('Zero credits : ',student_info[student_info['studied_credits'] == 0].size)

----------- Student Final Results -------------
Pass           12361
Withdrawn      10156
Fail            7052
Distinction     3024
Name: final_result, dtype: int64
----------- Student previous credits -------------
60     16751
120     6328
30      3749
90      3144
180      830
150      769
240      220
210      169
75       108
70        96
270       45
130       38
45        36
135       35
100       30
300       29
80        26
40        22
160       14
105       13
330       12
110       11
195       11
165        9
220        9
50         7
145        7
170        6
190        6
360        6
       ...  
140        5
85         4
200        3
95         3
55         3
225        3
250        3
280        3
205        2
310        2
400        2
430        1
480        1
370        1
175        1
65         1
235        1
355        1
420        1
325        1
155        1
540        1
390        1
630        1
215        1
255        1
345        1
585        1
315        1
655 

***Insights from Above Data***
- max credits for a student are 655 and minimum value is 30 credits
- most of the students in the dataset are having 60 credits
- There are no students with zero credits

In [86]:
#get student past credits for passing students
passing_student_info = student_info[student_info['final_result']=='Pass']
print('----------- Student previous credits -- PASS-------------')
#print(passing_student_info['studied_credits'].value_counts())
print('Minimum_Credits(PASS)',passing_student_info['studied_credits'].min())
print('Maximum_Credits(PASS)',passing_student_info['studied_credits'].max())

#get student past credits for withdrawing students
withdrawn_student_info = student_info[student_info['final_result']=='Withdrawn']
print('----------- Student previous credits -- WITHDRAWN -------------')
#print(withdrawn_student_info['studied_credits'].value_counts())
print('Minimum_Credits(Withdrawn)',withdrawn_student_info['studied_credits'].min())
print('Maximum_Credits(Withdrawn)',withdrawn_student_info['studied_credits'].max())


#get student past credits for Failing students
fail_student_info = student_info[student_info['final_result']=='Fail']
print('----------- Student previous credits -- FAIL -------------')
#print(fail_student_info['studied_credits'].value_counts())
print('Minimum_Credits(Fail)',fail_student_info['studied_credits'].min())
print('Maximum_Credits(Fail)',fail_student_info['studied_credits'].max())



----------- Student previous credits -- PASS-------------
Minimum_Credits(PASS) 30
Maximum_Credits(PASS) 630
----------- Student previous credits -- WITHDRAWN -------------
Minimum_Credits(Withdrawn) 30
Maximum_Credits(Withdrawn) 655
----------- Student previous credits -- FAIL -------------
Minimum_Credits(Fail) 30
Maximum_Credits(Fail) 360


In [80]:
#get the details of the student with highest_Credits
student_info[student_info['studied_credits'] == 655]

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
2778,BBB,2013J,363151,M,North Western Region,Lower Than A Level,10-20,35-55,0,655,Y,Withdrawn


In [87]:
#get the details of the student with highest_Credits
student_info[student_info['studied_credits'] == 630]

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
10378,CCC,2014B,1474869,F,South Region,A Level or Equivalent,90-100%,35-55,0,630,Y,Pass


### Lets analyze the data for one course -- AAA - 2013J
say course - AAA for 2013J session starting in october for 268 days

get the number of students registered for the course - 383 (from above group by statement)

In [93]:
# pass fail for Course- AAA
students_AAA = student_info[(student_info['code_module'] == 'AAA') & (student_info['code_presentation'] == '2013J')]
students_AAA['final_result'].value_counts()

Pass           258
Withdrawn       60
Fail            45
Distinction     20
Name: final_result, dtype: int64

In [94]:
#get educational_qualifications of students in course
students_AAA['highest_education'].value_counts()

A Level or Equivalent          194
HE Qualification                98
Lower Than A Level              83
Post Graduate Qualification      8
Name: highest_education, dtype: int64

In [120]:
#Get Assessments
assessments_AAA = assessments[(assessments['code_module'] == 'AAA') & (assessments['code_presentation'] == '2013J')]
assessments_AAA

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0
5,AAA,2013J,1757,Exam,?,100.0


In [136]:
#Get Student Assessment Details
#Student_assessments_AAA = student_assessment[student_assessment['id_assessment']]

f = lambda x : x in assessments_AAA['id_assessment'].tolist()
student_assessments_AAA = student_assessment[student_assessment['id_assessment'].apply(f)]
#student_assessments_AAA['id_student'].nunique() #--- 365
student_assessments_AAA['id_assessment'].value_counts() 
#--- final Exam Attempt is not given
#--- all the students did not submit all assignments
#--- there are some students who have not submitted the assignments at all -- may be withdrawn from course
#student_assessments_AAA

1752    359
1753    342
1754    331
1755    303
1756    298
Name: id_assessment, dtype: int64

In [151]:
# Check VLE interaction for course - AAA
vle_AAA=vle[(vle['code_module'] == 'AAA') & (vle['code_presentation'] == '2013J')]
vle_AAA['activity_type'].value_counts() #--- total 211 rows
#vle_AAA.head()
#vle_AAA[vle_AAA['id_site'] == 546614] #------ Home Page -- 1st Most Visited
#vle_AAA[vle_AAA['id_site'] == 546654] #------ forumng -- 2nd Most Visited

resource         95
oucontent        68
url              18
forumng          15
subpage           6
dataplus          4
oucollaborate     2
glossary          2
homepage          1
Name: activity_type, dtype: int64

In [137]:
# get student_interactions with vle for the course
student_vle_AAA = student_vle[(student_vle['code_module'] == 'AAA')&(student_vle['code_presentation'] == '2013J')]
#student_vle_AAA['id_student'].nunique() #---378
student_vle_AAA.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [149]:
#-- group by id-sites to get which has more entries in data
student_vle_AAA['id_site'].value_counts() #

#-- get which id_site has more clicks in data
#student_vle_AAA.groupby(['id_site','sum_click']).size()

546614    33429
546654    10170
546650     8668
546879     8483
546876     8176
546657     7625
546883     7455
546655     4694
546644     4540
546651     3853
546652     3720
546871     3488
546656     3482
546732     2355
546645     2336
546649     2095
546681     1942
546692     1887
546734     1799
546667     1792
546719     1766
546684     1583
546647     1505
546662     1482
546713     1466
546672     1388
546671     1374
546691     1358
546731     1313
546668     1297
          ...  
546891        4
546988        4
546933        3
546923        3
546925        3
546992        3
547040        2
547003        2
546942        2
547014        2
547021        2
546994        2
546679        1
547006        1
547031        1
547002        1
547038        1
546919        1
547027        1
546985        1
547025        1
547034        1
547026        1
547023        1
546914        1
547042        1
546906        1
547012        1
547019        1
546955        1
Name: id_site, Length: 2